In [3]:
!pip install numpy
!pip install pyopenms
!pip install biopython
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 5.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.8 MB/s eta 0:00:00a 0:00:01


In [ ]:
# # -*- coding: utf-8 -*-
# # Kamil's code
# # неактуальная ячейка с кодом 

# import pandas as pd
# import os

# # specifying the path to csv files
# path = "./requests/KJE0002"
 
# # csv files in the path
# file_paths = glob.glob(path + "/*.xlsx")

# def is_valid_sequence(seq):
#     if pd.isna(seq):
#         return False
#     valid_chars = set("ATGCUYRSWKMBDHVN.-")
#     return all(c in valid_chars for c in str(seq))

# def process_excel_files(file_paths):
#     # Список для хранения обработанных DataFrame из каждого файла
#     dfs_list = []

#     for file_path in file_paths:
#         # Чтение файла Excel
#         df = pd.read_excel(file_path)

#         # Получение имени самого левого столбца
#         leftmost_column = df.columns[0]

#         # Фильтрация строк согласно заданным условиям
#         df = df[df[leftmost_column].apply(is_valid_sequence)]

#         # Добавление обработанного DataFrame в список
#         dfs_list.append(df)

#     # Конкатенация всех обработанных DataFrame по строкам
#     final_df = pd.concat(dfs_list, axis=0, join='outer', ignore_index=True)

#     # Сохранение результата в новый файл Excel
#     final_df.to_excel("concatenated_file.xlsx", index=False)

# # Пути к файлам Excel для обработки (замените на свои пути)
# file_paths = ["/Users/MrZaynullin/Downloads/KJE0002/стрептококи.xlsx",
#               "/Users/MrZaynullin/Downloads/KJE0002/Общий файл праймеров_NIKITEEV.xlsx",
#               "/Users/MrZaynullin/Downloads/KJE0002/Копия ФОРМА ЗАКАЗА 2 Калиш - Сергей Калиш.xlsx",
#               "/Users/MrZaynullin/Downloads/KJE0002/ЗАКАЗ-Ковальчук_сентябрь_ 2023 - Sveta Kovalchuk.xlsx",
#               "/Users/MrZaynullin/Downloads/KJE0002/ФОРМА ЗАКАЗА - Николай Постика.xlsx",
#               "/Users/MrZaynullin/Downloads/KJE0002/ФОРМА ЗАКАЗА - Николай Постика (1).xlsx",
#               "/Users/MrZaynullin/Downloads/KJE0002/ФОРМА ЗАКАЗА - Tatiana Semashko.xlsx",
#               "/Users/MrZaynullin/Downloads/KJE0002/PAM_CAS_FOR_OVFF376.xlsx"
#               ]
# # Запуск функции обработки
# process_excel_files(file_paths)

In [2]:

# # importing the required modules
# import glob
# # specifying the path to csv files
# path = "./requests/KJE0002"
 
# # csv files in the path
# file_list = glob.glob(path + "/*.xlsx")

# file_list 

['./requests/KJE0002/ФОРМА ЗАКАЗА - Николай Постика.xlsx',
 './requests/KJE0002/Копия ФОРМА ЗАКАЗА 2 Калиш - Сергей Калиш.xlsx',
 './requests/KJE0002/ФОРМА ЗАКАЗА - Николай Постика (1).xlsx',
 './requests/KJE0002/стрептококи.xlsx',
 './requests/KJE0002/PAM_CAS_FOR_OVFF376.xlsx',
 './requests/KJE0002/ФОРМА ЗАКАЗА - Tatiana Semashko.xlsx',
 './requests/KJE0002/ЗАКАЗ-Ковальчук_сентябрь_ 2023 - Sveta Kovalchuk.xlsx',
 './requests/KJE0002/Общий файл праймеров_NIKITEEV.xlsx']

In [55]:
#######################
###  Запускать тут ####
#######################

import glob
import pandas as pd
import os

# CHANGE THIS
EXPERIMENT_NAME = "KJE0006"
# specifying the path to csv files
path = f"./requests/{EXPERIMENT_NAME}"
 
# csv files in the path
file_paths = glob.glob(path + "/*.xlsx")

def is_valid_sequence(seq):

    if pd.isna(seq):
        return False
    valid_chars = set("ATGCUYRSWKMBDHVN.-")
    return all(c in valid_chars for c in str(seq))

def process_excel_files(file_paths):
    # Список для хранения обработанных DataFrame из каждого файла
    dfs_list = []

    for file_path in file_paths:
        # Чтение файла Excel
        df = pd.read_excel(file_path, header=None)

        if len(df.columns)==1:
            df = df[0].str.split(',', expand=True)

        if len(df.columns)<4:
            return print("ERROR IN INPUT FILE")

        # df curation
        for column in df.columns:
            df[column] = df[column].apply(lambda x: unicodedata.normalize('NFC', str(x)))
        #  pandas' Series.str.strip() method
        df = df.apply(lambda x: x.str.strip())
        df[0] = df[0].apply(lambda x: x.upper())

        # Получение имени самого левого столбца
        leftmost_column = df.columns[0]
        print(file_path, len(df))
        # Фильтрация строк согласно заданным условиям
        df = df[df[leftmost_column].apply(is_valid_sequence)]
        print(file_path, len(df))
        # Добавление обработанного DataFrame в список
        dfs_list.append(df)
        
    return dfs_list
        
dfs_list=process_excel_files(file_paths)



./requests/KJE0006/Заявка на праймеры секв HC_LC - Viktor P.V..xlsx 6
./requests/KJE0006/Заявка на праймеры секв HC_LC - Viktor P.V..xlsx 3
./requests/KJE0006/ФОРМА ЗАКАЗА - Николай Постика 1.xlsx 4
./requests/KJE0006/ФОРМА ЗАКАЗА - Николай Постика 1.xlsx 1
./requests/KJE0006/Вибриофаг FS-1_V3 - Глеб Фисунов.xlsx 287
./requests/KJE0006/Вибриофаг FS-1_V3 - Глеб Фисунов.xlsx 287
./requests/KJE0006/DNA oligos Vadim  09292023 - Vadim Molodtsov.xlsx 177
./requests/KJE0006/DNA oligos Vadim  09292023 - Vadim Molodtsov.xlsx 177
./requests/KJE0006/из KJE0001.xlsx 54
./requests/KJE0006/из KJE0001.xlsx 54
./requests/KJE0006/ФОРМА ЗАКАЗА 2.xlsx 5
./requests/KJE0006/ФОРМА ЗАКАЗА 2.xlsx 2
./requests/KJE0006/PAM_CAS_FOR_OVFF376.xlsx 87
./requests/KJE0006/PAM_CAS_FOR_OVFF376.xlsx 87
./requests/KJE0006/ФОРМА ЗАКАЗА 1 .xlsx 5
./requests/KJE0006/ФОРМА ЗАКАЗА 1 .xlsx 2


In [58]:
# Конкатенация всех обработанных DataFrame по строкам
final_df = pd.concat(dfs_list, axis=0, join='outer', ignore_index=True)

if len(final_df)>384 & len(final_df)<768:
    
    df_A = final_df[:384]
    df_B = final_df[384:]
    
    # Сохранение результата в новый файл CSV
    df_A.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_A.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
    # Сохранение результата в новый файл Excel
    df_A.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_A.xlsx", index=False, header = None, encoding = "UTF-8")
    
     # Сохранение результата в новый файл CSV
    df_B.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_B.csv", index=False, sep = ',', header = None, encoding = "UTF-8")
    # Сохранение результата в новый файл Excel
    df_B.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file_plate_B.xlsx", index=False, header = None, encoding = "UTF-8")

else:

    # Сохранение результата в новый файл CSV
    final_df.to_csv(f"./result_input/{EXPERIMENT_NAME}_concatenated_file.csv", index=False, sep = ',', header = None, encoding = "UTF-8")

    # Сохранение результата в новый файл Excel
    final_df.to_excel(f"./result_input/{EXPERIMENT_NAME}_concatenated_file.xlsx", index=False, header = None, encoding = "UTF-8")

['./requests/KJE0006/Заявка на праймеры секв HC_LC - Viktor P.V..xlsx',
 './requests/KJE0006/ФОРМА ЗАКАЗА - Николай Постика 1.xlsx',
 './requests/KJE0006/Вибриофаг FS-1_V3 - Глеб Фисунов.xlsx',
 './requests/KJE0006/DNA oligos Vadim  09292023 - Vadim Molodtsov.xlsx',
 './requests/KJE0006/ФОРМА ЗАКАЗА 2.xlsx',
 './requests/KJE0006/ФОРМА ЗАКАЗА 1 .xlsx']

SyntaxError: 'return' outside function (1237529054.py, line 11)